In [1]:
from fbprophet import Prophet
import pandas as pd

In [2]:
# READ IN THE DATA
from google.colab import files
files.upload()

Saving housing_data.csv to housing_data.csv


In [3]:
full_data = pd.read_csv("housing_data.csv")

In [4]:
# THE DATES ARE CURRENTLY LOCATED IN THE DATA AS COLUMNS (NOT GOOD!!)
# WE WANT ONE COLUMN NAMED 'DATES' THAT HOLDS EACH INSTANCE FOR EACH DATE FOR EACH ZIP CODE
# GO FROM A WIDE DATASET TO A LONG DATASET
# ID_VARS WILL BE LEFT ALONE
# THE REMAINING COLUMNS (THE DATES IN THIS CASE) WILL BE MELTED INTO ONE COLUMN WHICH IS NAMED 'DATES'
# THE VALUES THAT ARE LOCATED UNDER THE DATE COLUMNS IN THE MEDIAN HOUSING VALUE, WE WANT TO KEEP THESE TOGETHER
# SO VAR_NAME IS THE NAME OF THE NEW COLUMN BEING MELTED AND VALUE_NAME IS THE VALUES WHICH WERE HELD WITH THE VAR_NAME 
full_data_melted = full_data.melt(id_vars = ["RegionID", 
                                           "SizeRank", 
                                           "RegionName", 
                                           "RegionType", 
                                           "StateName", 
                                           "State",
                                           "City",
                                           "Metro",
                                           "CountyName"],
                                var_name = "Date",
                                value_name = "MedianHousingValue")
# CHANGE THE DATE COLUMN TO DATETIME TYPE
full_data_melted["Date"] = pd.to_datetime(full_data_melted["Date"])


In [5]:
# GET ALL THE ZIP CODES FROM THE BUSINESS INSIDER TOP 25 CITIES TO LIVE IN IN THE US
from functools import reduce

city1 = full_data_melted[full_data_melted["City"] == "Melbourne"]
city2 = full_data_melted[full_data_melted["City"] == "Salt Lake City"]
city3 = full_data_melted[full_data_melted["City"] == "Portland"]
city4 = full_data_melted[full_data_melted["City"] == "Greenville"]
city5 = full_data_melted[full_data_melted["City"] == "Dallas"]
city6 = full_data_melted[full_data_melted["City"] == "Charlotte"]
city7 = full_data_melted[full_data_melted["City"] == "Sarasota"]
city8 = full_data_melted[full_data_melted["City"] == "Boise"]
city9 = full_data_melted[full_data_melted["City"] == "Asheville"]
city10 = full_data_melted[full_data_melted["City"] == "Nashville"]
city11 = full_data_melted[full_data_melted["City"] == "San Jose"]
city12 = full_data_melted[full_data_melted["City"] == "Grand Rapids"]
city13 = full_data_melted[full_data_melted["City"] == "Madison"]
city14 = full_data_melted[full_data_melted["City"] == "Huntsville"]
city15 = full_data_melted[full_data_melted["City"] == "Raleigh"]
city16 = full_data_melted[full_data_melted["City"] == "Seattle"]
city17 = full_data_melted[full_data_melted["City"] == "San Fransisco"]
city18 = full_data_melted[full_data_melted["City"] == "Minneapolis"]
city19 = full_data_melted[full_data_melted["City"] == "Des Moines"]
city20 = full_data_melted[full_data_melted["City"] == "Fayetteville"]
city21 = full_data_melted[full_data_melted["City"] == "Colorado Springs"]
city22 = full_data_melted[full_data_melted["City"] == "Denver"]
city23 = full_data_melted[full_data_melted["City"] == "Austin"]

# CREATE A LIST OF THE DATAFRAMES TO MERGE (ONE FOR EACH CITY COLLECTED ABOVE)
all_cities = [city1, city2, city3, city4, city5, city6, city7, city8, city9, city10, city11, city12, city13, city14, city15, city16, city17, city18, city19, city20, city21, city22, city23]
# MERGE ALL OF THE DATAFRAMES, OUTER SO WE INCLUDE ALL THE DATA
best_cities = reduce(lambda left, right: pd.merge(left, right, how = "outer"), all_cities)
# DROP THE NA VALUES 
best_cities.dropna(axis = 0, how = "any", inplace = True)


In [6]:
zips = []
predictions = []
actual = []
differences = []

def predict_zip(zip_code):
  # CREATE NEW DF WITH ONLY THE DATA FROM SPECIFIC ZIP CODE 
  df = best_cities[best_cities["RegionID"] == zip_code]
  # CREATE TRAINING AND TESTING SETS (TRAIN = 1996-2019, TEST = 2020)
  all_train = df[pd.DatetimeIndex(df["Date"]).year != 2020]
  all_test = df[pd.DatetimeIndex(df["Date"]).year == 2020]

  # CREATE EMPTY DATAFRAME THAT WILL BE USED TO CREATE DF SUITABLE FOR PROPHET
  # DO SO FOR BOTH THE TRAIN AND TEST
  train = pd.DataFrame()
  train["ds"] = all_train["Date"]
  train['y'] = all_train["MedianHousingValue"]

  test = pd.DataFrame()
  test["ds"] = all_test["Date"]
  test["y"] = all_test["MedianHousingValue"]

  # INITIATE, TRAIN, AND PREDICT USING THE ABOVE DATAFRAMES
  m = Prophet()
  m.fit(train)
  results = m.predict(test)

  # CALCULATE THE DIFFERENCE BETWEEN THE MAX PREDICTION AND THE MAX ACTUAL VALUE
  # APPEND ALL OF THE DIFFERENT LISTS WITH THE RESULTS FROM PROPHET 
  difference = (results["yhat"].max() - all_train["MedianHousingValue"].max())
  zips.append(zip_code)
  predictions.append(results["yhat"].max())
  actual.append(all_train["MedianHousingValue"].max())
  differences.append(difference)

  return zips
  return predictions
  return actual
  return difference
  


In [ ]:
# CREATE A LIST OF EACH UNIQUE ZIP CODE IN THE DATAFRAME
new_zips = best_cities["RegionID"].unique().tolist()

# RUN THE ABOVE FUNCTION FOR EACH UNIQUE ZIP CODE IN THE LIST 
for zip_code in new_zips:
  predict_zip(zip_code)

# TAKE APPROXIMATELY 20 MINUTES TO FINISH RUNNING!!!!!!

In [12]:
# CREATE DICTIONARY OF THE RESULTS FROM PROPHET 
# WILL BE MADE INTO A DATAFRAME LATER
p_results = {}
p_results["Zip"] = zips
p_results["Actual"] = actual
p_results["Prediction"] = predictions
p_results["Difference"] = differences

In [ ]:
results_df = pd.DataFrame(p_results, columns = p_results.keys())
#results_df.head(10)

In [38]:
# THIS FUNCTION TAKES A SERIES, LIST, ETC AND SPITS OUT A SPECIFIC NUMBER OF VALUES THAT ARE CLOSEST TO A VALUE YOU CHOOSE
def closest_value(sample, number, return_num):
    return sorted(sample, key=lambda i: abs(i - number))[:return_num]

In [63]:
# FIND THE 10 VALUES IN THE COLUMN THAT ARE CLOSEST TO 0
close_values = closest_value(results_df["Difference"], 0, 10)
#print(close_values)

[4.138011992676184, -5.248831955497735, 52.43917267775396, 69.26235530945996, 70.37528466351796, 78.53466826223303, -83.00751191485324, 85.68060767033603, -129.77141264948295, -135.0369090141321]


In [52]:
# DISPLAY THE ZIP CODES OF THE VALUES THAT WERE CLOSELY PREDICTED
one = results_df["Zip"][results_df["Difference"] == close_values[0]]
two = results_df["Zip"][results_df["Difference"] == close_values[1]]
three = results_df["Zip"][results_df["Difference"] == close_values[2]]
four = results_df["Zip"][results_df["Difference"] == close_values[3]]
five = results_df["Zip"][results_df["Difference"] == close_values[4]]
six = results_df["Zip"][results_df["Difference"] == close_values[5]]
seven = results_df["Zip"][results_df["Difference"] == close_values[6]]
eight = results_df["Zip"][results_df["Difference"] == close_values[7]]
nine = results_df["Zip"][results_df["Difference"] == close_values[8]]
ten = results_df["Zip"][results_df["Difference"] == close_values[9]]

print(one)
print(two)
print(three)
print(four)
print(five)
print(six)
print(seven)
print(eight)
print(nine)
print(ten)

502    92628
Name: Zip, dtype: int64
160    89603
Name: Zip, dtype: int64
292    73627
Name: Zip, dtype: int64
68    69533
Name: Zip, dtype: int64
305    73619
Name: Zip, dtype: int64
298    76632
Name: Zip, dtype: int64
125    69676
Name: Zip, dtype: int64
492    92666
Name: Zip, dtype: int64
382    80287
Name: Zip, dtype: int64
310    73629
Name: Zip, dtype: int64


In [53]:
# USING THE ZIP CODES ABOVE, DISPLAY THE CITY THAT EACH ZIP CODE IS ASSOCIATED WITH 
# NOT NECCESSARY BUT HELPS WITH EXPLAINING THE RESULTS AND IS INTERESTING 
a = full_data["City"][full_data["RegionID"] == 92628]
b = full_data["City"][full_data["RegionID"] == 89603]
c = full_data["City"][full_data["RegionID"] == 73627]
d = full_data["City"][full_data["RegionID"] == 69533]
e = full_data["City"][full_data["RegionID"] == 73619]
f = full_data["City"][full_data["RegionID"] == 76632]
g = full_data["City"][full_data["RegionID"] == 69676]
h = full_data["City"][full_data["RegionID"] == 92666]
i = full_data["City"][full_data["RegionID"] == 80287]
j = full_data["City"][full_data["RegionID"] == 73629]

print(a)
print(b)
print(c)
print(d)
print(e)
print(f)
print(h)
print(i)
print(j)

2642    Austin
Name: City, dtype: object
26964    Charlotte
Name: City, dtype: object
18509    Madison
Name: City, dtype: object
241    Greenville
Name: City, dtype: object
3287    Huntsville
Name: City, dtype: object
14832    Huntsville
Name: City, dtype: object
4222    Austin
Name: City, dtype: object
7828    Des Moines
Name: City, dtype: object
6843    Huntsville
Name: City, dtype: object


In [58]:
# FUNCTION THAT FINDS THE 10 MAXIMUM VALUES IN THE PREDICTIONS LIST 
def top_10_pred(list1, N = 10): 
    final_list = [] 
  
    for i in range(0, N):  
        max1 = 0
          
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j]; 
                  
        list1.remove(max1); 
        final_list.append(max1) 
          
    print(final_list)

In [60]:
top_10_pred(predictions)

[1435302.7059097372, 1319239.4496632023, 1311490.5378642273, 1304337.122329982, 1281646.389659733, 1277400.5963615584, 1272702.4443157332, 1268641.2967212552, 1264957.9083365323, 1245836.0667833125]


In [61]:
# DISPLAY THE ZIP CODE OF THE 10 HIGHEST PREDICTIONS
aa = results_df["Zip"][results_df["Prediction"] == 1435302.7059097372]
bb = results_df["Zip"][results_df["Prediction"] == 1319239.4496632023]
cc = results_df["Zip"][results_df["Prediction"] == 1311490.5378642273]
dd = results_df["Zip"][results_df["Prediction"] == 1304337.122329982]
ee = results_df["Zip"][results_df["Prediction"] == 1281646.389659733]
ff = results_df["Zip"][results_df["Prediction"] == 1277400.5963615584]
gg = results_df["Zip"][results_df["Prediction"] == 1272702.4443157332]
hh = results_df["Zip"][results_df["Prediction"] == 1268641.2967212552]
ii = results_df["Zip"][results_df["Prediction"] == 1264957.9083365323]
jj = results_df["Zip"][results_df["Prediction"] == 1245836.0667833125]

print(aa)
print(bb)
print(cc)
print(dd)
print(ee)
print(ff)
print(gg)
print(hh)
print(ii)
print(jj)

224    97992
Name: Zip, dtype: int64
247    97914
Name: Zip, dtype: int64
334    99569
Name: Zip, dtype: int64
338    99562
Name: Zip, dtype: int64
230    97996
Name: Zip, dtype: int64
156    69770
Name: Zip, dtype: int64
231    98002
Name: Zip, dtype: int64
344    99579
Name: Zip, dtype: int64
236    97986
Name: Zip, dtype: int64
234    98000
Name: Zip, dtype: int64


In [62]:
# USING THE ZIP CODES ABOVE, DISPLAY THE CITY 
# NOT NECCESSARY 
aaa = full_data["City"][full_data["RegionID"] == 97992]
bbb = full_data["City"][full_data["RegionID"] == 97914]
ccc = full_data["City"][full_data["RegionID"] == 99569]
ddd = full_data["City"][full_data["RegionID"] == 99562]
eee = full_data["City"][full_data["RegionID"] == 97996]
fff = full_data["City"][full_data["RegionID"] == 69770]
ggg = full_data["City"][full_data["RegionID"] == 98002]
hhh = full_data["City"][full_data["RegionID"] == 99597]
iii = full_data["City"][full_data["RegionID"] == 97986]
jjj = full_data["City"][full_data["RegionID"] == 98000]

print(aaa)
print(bbb)
print(ccc)
print(ddd)
print(eee)
print(fff)
print(hhh)
print(iii)
print(jjj)

1435    San Jose
Name: City, dtype: object
18688    San Jose
Name: City, dtype: object
610    Seattle
Name: City, dtype: object
1534    Seattle
Name: City, dtype: object
2643    San Jose
Name: City, dtype: object
9904    Charlotte
Name: City, dtype: object
4126    Burien
Name: City, dtype: object
3972    San Jose
Name: City, dtype: object
3700    San Jose
Name: City, dtype: object
